In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import time
import re
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

Mounted at /content/drive


读取TFIDF和SPEC的方式：
https://github.com/sciosci/AFT-MAG

https://zenodo.org/records/4917086

!pip3 install zenodo_get

wget https://zenodo.org/records/4917086/files/paper_specter_0.pkl

!zenodo_get 4917086 -g "paper_specter_1.pkl"

In [ ]:
Retention_=pd.read_csv('/content/drive/MyDrive/academic_tree/results/postdoc_retention.csv')

In [ ]:
Retention_=pd.read_csv('/content/drive/MyDrive/academic_tree/results/phd/phdtomentorauhtor.csv')

In [ ]:
Retention_.columns

Index(['CID', 'MenteeID', 'MentorID', 'MentorshipType', 'InstitutionMAGID',
       'StartYear', 'StopYear', 'Diff', 'retention', 'retention2',
       'MenteepostdocPapers', 'MenteePublishedPapersbefore',
       'MenteePaperscountbefore', 'MentorPublishedPapersbefore',
       'MentorPublishedPapersduring', 'OverlapCountduring',
       'menteepaperscount_d', 'MentorPapersduringcount',
       'MenteeColduringRatio', 'MentorColduringRatio', 'mentee_similarity_nor',
       'mentor_similarity_nor', 'cross_similarity_nor', 'Citation_Count',
       'C10', 'C5', 'Team_Size', 'Institution_Count', 'Disruption',
       'NIH_Count', 'NSF_Count', 'Atyp_10pct_Z', 'Atyp_Median_Z',
       'Patent_Count', 'SB_B', 'SB_T', 'Reference_Count',
       'mentorpaperscount_b', 'mentor_citation_count_b', 'mentor_team_size_b',
       'mentee_citation_count_before', 'ResearchArea', 'MAGAuthorID', 'gender',
       'race', 'JSD', 'JSD_sub', 'JSD_before', 'Hit_1pct_max', 'Hit_5pct_max',
       'Hit_10pct_max', 'C_f_m

In [ ]:
import ast
def safe_convert(x):
    if isinstance(x, list):  # 已经是列表，直接返回
        return x
    elif isinstance(x, str):  # 如果是字符串，尝试解析
        try:
            return ast.literal_eval(x)
        except:
            return []  # 解析失败则返回空列表
    else:  # 其他情况（如 NaN、数字等）
        return []
Retention_['MenteePublishedPapers'] = Retention_['MenteePublishedPapers'].apply(safe_convert)
Retention_['MentorPublishedPapers'] = Retention_['MentorPublishedPapers'].apply(safe_convert)
Retention_['MentorPublishedPapers_5'] = Retention_['MentorPublishedPapers_5'].apply(safe_convert)

In [ ]:
#postdoc
import ast
def safe_convert(x):
    if isinstance(x, list):  # 已经是列表，直接返回
        return x
    elif isinstance(x, str):  # 如果是字符串，尝试解析
        try:
            return ast.literal_eval(x)
        except:
            return []  # 解析失败则返回空列表
    else:  # 其他情况（如 NaN、数字等）
        return []
Retention_['MenteepostdocPapers'] = Retention_['MenteepostdocPapers'].apply(safe_convert)
Retention_['MenteePublishedPapersbefore']= Retention_['MenteePublishedPapersbefore'].apply(safe_convert)
Retention_['MentorPublishedPapersbefore'] = Retention_['MentorPublishedPapersbefore'].apply(safe_convert)
Retention_['MentorPublishedPapersduring']= Retention_['MentorPublishedPapersduring'].apply(safe_convert)
Retention_['MentorPublishedPapers_5']= Retention_['MentorPublishedPapers_5'].apply(safe_convert)

In [ ]:
from collections import Counter
def count_unique_paper_ids(data_column):
    # 展平所有列表并统计
    all_papers = []
    empty_count = 0

    for paper_list in data_column:
        if isinstance(paper_list, list) and len(paper_list) > 0:
            all_papers.extend(paper_list)
        else:
            empty_count += 1

    # 计算唯一值
    unique_count = len(set(all_papers))
    frequency = Counter(all_papers)

    return unique_count, frequency, empty_count

# 执行计算
unique_count, frequency_dict, empty_lists = count_unique_paper_ids(Retention_['MentorPublishedPapers'])

print(f"总列表数量: {len(Retention_['MentorPublishedPapers'])}")
print(f"空列表数量: {empty_lists} (占比 {empty_lists/len(Retention_['MentorPublishedPapers'])*100:.2f}%)")
print(f"唯一PaperID总数: {unique_count}")
print(f"出现频率最高的5个PaperID: {frequency_dict.most_common(5)}")


总列表数量: 423381
空列表数量: 118030 (占比 27.88%)
唯一PaperID总数: 1852999
出现频率最高的5个PaperID: [(2118550542, 96), (2108812250, 87), (2247766769, 60), (1495086940, 57), (2332750110, 56)]


In [ ]:
unique_count, frequency_dict, empty_lists = count_unique_paper_ids(Retention_['MentorPublishedPapers_5'])

print(f"总列表数量: {len(Retention_['MentorPublishedPapers_5'])}")
print(f"空列表数量: {empty_lists} (占比 {empty_lists/len(Retention_['MentorPublishedPapers_5'])*100:.2f}%)")
print(f"唯一PaperID总数: {unique_count}")
print(f"出现频率最高的5个PaperID: {frequency_dict.most_common(5)}")

总列表数量: 423381
空列表数量: 109079 (占比 25.76%)
唯一PaperID总数: 2234022
出现频率最高的5个PaperID: [(2108812250, 139), (2118550542, 122), (563944295, 103), (2332750110, 81), (1934211786, 79)]


In [ ]:
def extract_unique_paper_ids(data_column):
    unique_ids = set()
    for paper_list in data_column:
        # 跳过非列表项和空列表
        if not isinstance(paper_list, list) or len(paper_list) == 0:
            continue
        unique_ids.update(paper_list)
    return unique_ids
mentee_paper_ids = extract_unique_paper_ids(Retention_['MenteePublishedPapers'])
print(len(mentee_paper_ids))
mentor_paper_ids = extract_unique_paper_ids(Retention_['MentorPublishedPapers_5'])
print(len(mentor_paper_ids))
all_unique_paper_ids = mentee_paper_ids | mentor_paper_ids
print(len(all_unique_paper_ids))
print(f"合并后共有 {len(all_unique_paper_ids)} 个唯一PaperID")

1169382
2234022
3073204
合并后共有 3073204 个唯一PaperID


In [ ]:
def extract_unique_paper_ids(data_column):
    unique_ids = set()
    for paper_list in data_column:
        # 跳过非列表项和空列表
        if not isinstance(paper_list, list) or len(paper_list) == 0:
            continue
        unique_ids.update(paper_list)
    return unique_ids
mentee_paper_ids = extract_unique_paper_ids(Retention_['MenteepostdocPapers'])
print(len(mentee_paper_ids))
mentor_paper_ids = extract_unique_paper_ids(Retention_['MentorPublishedPapersbefore'])
print(len(mentor_paper_ids))
all_unique_paper_ids = mentee_paper_ids | mentor_paper_ids
print(len(all_unique_paper_ids))
print(f"合并后共有 {len(all_unique_paper_ids)} 个唯一PaperID")

219411
388148
563349
合并后共有 563349 个唯一PaperID


In [ ]:
def extract_unique_paper_ids(data_column):
    unique_ids = set()
    for paper_list in data_column:
        # 跳过非列表项和空列表
        if not isinstance(paper_list, list) or len(paper_list) == 0:
            continue
        unique_ids.update(paper_list)
    return unique_ids
mentee_paper_ids = extract_unique_paper_ids(Retention_['MenteepostdocPapers'])
print(len(mentee_paper_ids))
mentor_paper_ids = extract_unique_paper_ids(Retention_['MentorPublishedPapers_5'])
print(len(mentor_paper_ids))
all_unique_paper_ids = mentee_paper_ids | mentor_paper_ids
print(len(all_unique_paper_ids))
print(f"合并后共有 {len(all_unique_paper_ids)} 个唯一PaperID")

219411
529219
694537
合并后共有 694537 个唯一PaperID


In [ ]:
import gc
import os
def extract_target_vectors(target_ids, base_path, num_files=10):
    specter_dict = {}
    target_ids_str = {str(pid) for pid in target_ids}
    total_target = len(target_ids_str)

    # 遍历所有文件
    for i in range(num_files):
        file_path = f"{base_path}{i}.pkl"

        # 检查文件是否存在
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}，跳过")
            continue

        print(f"\n开始处理文件: {file_path}")
        found_in_this_file = 0
        with open(file_path, 'rb') as fin:
            unpickler = pickle.Unpickler(fin)
            try:
                # 使用无限循环遍历文件内容
                for _ in tqdm(iter(int, 1), desc="处理记录"):
                    pubid, vec = unpickler.load()
                    pubid_str = str(pubid)

                    # 检查是否为目标ID
                    if pubid_str in target_ids_str:
                        specter_dict[pubid_str] = vec
                        found_in_this_file += 1

            except EOFError:
                pass
            except Exception as e:
                print(f"处理文件时出错: {str(e)}")

        current_found = len(specter_dict)
        print(f"本文件找到: {found_in_this_file} 个向量")
        print(f"累计找到: {current_found} 个 ({current_found/total_target*100:.2f}%)")

        # 显式释放内存
        del unpickler
        gc.collect()

    total_found = len(specter_dict)
    completion_rate = (total_found / total_target) * 100

    return specter_dict, total_found, completion_rate

# 使用示例
if __name__ == "__main__":
    # 文件基础路径
    base_path = '/content/drive/MyDrive/academic_tree/data/spec/paper_specter_'

    # 执行处理
    specter_dict, total_found, completion_rate = extract_target_vectors(
        all_unique_paper_ids, base_path, num_files=10
    )

    # 打印最终结果
    print("\n" + "="*50)
    print(f"处理完成结果:")
    print(f"目标PaperID总数: {len(all_unique_paper_ids)}")
    print(f"成功找到的PaperID数量: {total_found}")
    print(f"完成率: {completion_rate:.2f}%")
    print("="*50)

    # 可选：保存结果
    #with open('/content/drive/MyDrive/academic_tree/data/spec/postdocretention.pkl', 'wb') as f:
       # pickle.dump(specter_dict, f)



开始处理文件: /content/drive/MyDrive/academic_tree/data/spec/paper_specter_0.pkl


处理记录: 1694701it [02:35, 10929.20it/s]


本文件找到: 39306 个向量
累计找到: 39306 个 (5.66%)

开始处理文件: /content/drive/MyDrive/academic_tree/data/spec/paper_specter_1.pkl


处理记录: 1694315it [01:32, 18290.66it/s]


本文件找到: 111311 个向量
累计找到: 150617 个 (21.69%)

开始处理文件: /content/drive/MyDrive/academic_tree/data/spec/paper_specter_2.pkl


...

本文件找到: 23670 个向量
累计找到: 656622 个 (94.54%)

开始处理文件: /content/drive/MyDrive/academic_tree/data/spec/paper_specter_8.pkl


处理记录: 1694170it [01:14, 22598.77it/s]


本文件找到: 22415 个向量
累计找到: 679037 个 (97.77%)

开始处理文件: /content/drive/MyDrive/academic_tree/data/spec/paper_specter_9.pkl


处理记录: 1694135it [02:26, 11549.94it/s]


本文件找到: 15500 个向量
累计找到: 694537 个 (100.00%)

处理完成结果:
目标PaperID总数: 694537
成功找到的PaperID数量: 694537
完成率: 100.00%


In [ ]:
specter=specter_dict.copy()

In [ ]:
len(specter)

694537

归一化

In [ ]:
for pid in specter:
    vec = specter[pid]
    norm = np.linalg.norm(vec)
    if norm > 0:
        specter[pid] = vec / norm

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [ ]:
def calc_phase_similarity(paper_list):
    # 处理无效输入
    if len(paper_list) < 2:
        return np.nan

    vectors = []
    for pid in paper_list:
        pid_str = str(pid)
        if pid_str in specter:
            vectors.append(specter[pid_str])


    sim_matrix = cosine_similarity(vectors)

    # 计算非对角线元素的平均值
    return np.mean(sim_matrix[np.triu_indices(len(sim_matrix), k=1)])

# 应用函数计算所有列表的相似度
'''
Retention_['mentee_similarity_spec'] = [calc_phase_similarity(papers) for papers in tqdm(Retention_['MenteePublishedPapers'])]
Retention_['mentor_similarity_spec'] = [calc_phase_similarity(papers) for papers in tqdm(Retention_['MentorPublishedPapers'])]
'''
# 应用函数计算所有列表的相似度
Retention_['mentee_similarity_spec'] = [calc_phase_similarity(papers) for papers in tqdm(Retention_['MenteepostdocPapers'])]
Retention_['mentor_similarity_spec'] = [calc_phase_similarity(papers) for papers in tqdm(Retention_['MentorPublishedPapersbefore'])]

100%|██████████| 30421/30421 [00:29<00:00, 1022.03it/s]


In [ ]:
Retention_[['mentee_similarity_nor','mentee_similarity_spec','mentor_similarity_nor','mentor_similarity_spec']].corr()

,mentee_similarity_nor,mentee_similarity_spec,mentor_similarity_nor,mentor_similarity_spec
mentee_similarity_nor,1.000000,0.603668,0.152908,0.049398
mentee_similarity_spec,0.603668,1.000000,0.057854,0.207801
mentor_similarity_nor,0.152908,0.057854,1.000000,0.588692
mentor_similarity_spec,0.049398,0.207801,0.588692,1.000000


In [ ]:
def calc_cross_similarity(mentee_papers, mentor_papers):
    mentee_vectors = []
    for pid in mentee_papers:
        pid_str = str(pid)
        if pid_str in specter_dict:
            mentee_vectors.append(specter_dict[pid_str])
    mentor_vectors = []
    for pid in mentor_papers:
        pid_str = str(pid)
        if pid_str in specter_dict:
            mentor_vectors.append(specter_dict[pid_str])
    # 检查是否有足够的向量进行计算
    if not mentee_vectors or not mentor_vectors:
        return np.nan
    # 转换为NumPy数组
    #mentee_matrix = np.array(mentee_vectors)
    #mentor_matrix = np.array(mentor_vectors)

    # 计算跨列表相似度矩阵
    sim_matrix = cosine_similarity(mentee_vectors, mentor_vectors)
    #sim_matrix = mentee_matrix.dot(mentor_matrix.T)  # 等价于余弦相似度（当向量归一化时）

    # 计算并返回所有配对相似度的均值
    return np.mean(sim_matrix)

# 为了提高大计算效率，添加进度条
tqdm.pandas()  # 启用pandas的进度条支持
'''
Retention_['Cross_Similarity2'] = Retention_.progress_apply(
    lambda row: calc_cross_similarity(row['MenteePublishedPapers'], row['MentorPublishedPapers']),
    axis=1
)
'''
#Retention_['cross_similarity_spec'] = Retention_.progress_apply(lambda row: calc_cross_similarity(row['MenteepostdocPapers'], row['MentorPublishedPapersbefore']),axis=1)
Retention_['cross_similarity_spec_5'] = Retention_.progress_apply(lambda row: calc_cross_similarity(row['MenteepostdocPapers'], row['MentorPublishedPapers_5']),axis=1)

100%|██████████| 30421/30421 [00:45<00:00, 672.34it/s]


In [ ]:
Retention_.columns

Index(['CID', 'MenteeID', 'MentorID', 'MentorshipType', 'InstitutionMAGID',
       'StartYear', 'StopYear', 'Diff', 'retention', 'retention2',
       'MenteepostdocPapers', 'MenteePublishedPapersbefore',
       'MenteePaperscountbefore', 'MentorPublishedPapersbefore',
       'MentorPublishedPapersduring', 'OverlapCountduring',
       'menteepaperscount_d', 'MentorPapersduringcount',
       'MenteeColduringRatio', 'MentorColduringRatio', 'mentee_similarity_nor',
       'mentor_similarity_nor', 'cross_similarity_nor', 'Citation_Count',
       'C10', 'C5', 'Team_Size', 'Institution_Count', 'Disruption',
       'NIH_Count', 'NSF_Count', 'Atyp_10pct_Z', 'Atyp_Median_Z',
       'Patent_Count', 'SB_B', 'SB_T', 'Reference_Count',
       'mentorpaperscount_b', 'mentor_citation_count_b', 'mentor_team_size_b',
       'mentee_citation_count_before', 'ResearchArea', 'MAGAuthorID', 'gender',
       'race', 'JSD', 'JSD_sub', 'JSD_before', 'Hit_1pct_max', 'Hit_5pct_max',
       'Hit_10pct_max', 'C_f_m

In [ ]:
Retention_.drop(['MentorPublishedPapers_5_y','MentorPublishedPapers_5_x'],axis=1,inplace=True)


In [ ]:
Retention_[['Cross_Similarity','Cross_Similarity3','Cross_Similarity2','cross_similarity_spec_5']].corr()

,Cross_Similarity,Cross_Similarity3,Cross_Similarity2,cross_similarity_spec_5
Cross_Similarity,1.000000,1.000000,0.545096,0.519285
Cross_Similarity3,1.000000,1.000000,0.545096,0.519285
Cross_Similarity2,0.545096,0.545096,1.000000,0.978981
cross_similarity_spec_5,0.519285,0.519285,0.978981,1.000000


In [ ]:
Retention_[['cross_similarity_nor','cross_similarity_spec']].corr()

In [ ]:
Retention_.to_csv('/content/drive/MyDrive/academic_tree/results/postdoc_retention.csv',index=False)

In [ ]:
Retention_.to_csv('/content/drive/MyDrive/academic_tree/results/phd/phdtomentorauhtor.csv',index=False)

In [ ]:
Retention_.columns

Index(['CID', 'MenteeID', 'MentorID', 'MentorshipType', 'InstitutionMAGID',
       'StartYear', 'StopYear', 'Diff', 'retention', 'retention2',
       'MenteePublishedPapers', 'MentorPublishedPapers', 'OverlapCount',
       'MenteePapersCount', 'MentorPapersCount', 'mentee_similarity',
       'mentor_similarity', 'Cross_Similarity', 'Citation_Count', 'C10', 'C5',
       'Team_Size', 'Institution_Count', 'Disruption', 'NIH_Count',
       'NSF_Count', 'Atyp_10pct_Z', 'Atyp_Median_Z', 'Patent_Count', 'SB_B',
       'SB_T', 'Reference_Count', 'ResearchArea', 'MAGAuthorID', 'gender',
       'race', 'JSD', 'JSD_sub', 'Hit_1pct_max_x', 'Hit_5pct_max_x',
       'Hit_10pct_max_x', 'C_f_mean', 'mentee_similarity2',
       'mentor_similarity2', 'Cross_Similarity2', 'mentee_similarity3',
       'mentor_similarity3', 'Cross_Similarity3', 'mentor_citation_count',
       'mentor_team_size', 'mentor_Hit_5pct', 'MentorPublishedPapersduring',
       'OverlapCountduring', 'MenteePapersduringcount',
      

link

In [ ]:
Retention_=pd.read_csv('/content/drive/MyDrive/academic_tree/results/phd/phdtomentorauhtor.csv')

In [ ]:
re_=Retention_[['CID','cross_similarity_spec_5']]

In [ ]:
re_

,CID,cross_similarity_spec_5
0,346,0.820586
1,1256,0.763695
2,3,0.817108
3,9624,NaN
4,38845,0.746228
...,...,...
30416,1784658,0.791805
30417,1784660,0.787098
30418,1784686,0.733305
30419,1784716,NaN


In [ ]:
re_.to_csv('/content/drive/MyDrive/academic_tree/cross_similarity_spec_5.csv',index=False)